In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#Importing and pre-processing the images
DATADIR = "Datasets/PetImages"
CATEGORIES = ["Cat", "Dog"]

training_data = []
IMG_SIZE = 70

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [4]:
print(len(training_data))

24946


In [5]:
#Randomizing the data
import random

random.shuffle(training_data)

In [6]:
X = []
y = []

In [7]:
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [9]:
X = X/255.0

In [ ]:
# Trying out different combinations of layers
dense_layers = [0,1,2]
layer_sizes = [64, 128, 256]
conv_layers = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f'dense{dense_layer}-layer_size{layer_size}-conv_layer{conv_layer}-{int(time.time())}'
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), activation="relu", input_shape = X.shape[1:]))
            model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3), activation="relu"))
                model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            model.add(Flatten())

            for l in range(dense_layer):
                model.add(Dense(layer_size, activation= "relu"))
                model.add(Dropout(0.2))

            model.add(Dense(1, activation= "sigmoid"))

            model.compile(loss="binary_crossentropy", 
                            optimizer="adam", 
                            metrics=["accuracy"])

            model.fit(X, y, batch_size=32, 
                            epochs=10, 
                            validation_split=0.3,
                            callbacks= [tensorboard])

In [10]:
#Best combination for accuracy: 85% achieved
#Convolutional layers=3, Dense layers=1, Layer size = 64

NAME = f'dense1-layer_size64-conv_layer3-{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
model = Sequential()

model.add(Conv2D(64, (3,3), activation="relu", input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(64, (3,3), activation="relu",)) 
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())

model.add(Dense(64, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation= "sigmoid"))

model.compile(loss="binary_crossentropy", 
                optimizer="adam", 
                metrics=["accuracy"])

model.fit(X, y, batch_size=32, 
                epochs=10, 
                validation_split=0.3,
                callbacks= [tensorboard])

2022-04-27 20:45:33.894673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 20:45:33.921290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 20:45:33.921491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 20:45:33.921866: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

Epoch 1/10


2022-04-27 20:45:35.739041: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303


546/546 [==============================] - 6s 8ms/step - loss: 0.6141 - accuracy: 0.6520 - val_loss: 0.5286 - val_accuracy: 0.7340
Epoch 2/10
546/546 [==============================] - 4s 7ms/step - loss: 0.5068 - accuracy: 0.7534 - val_loss: 0.4605 - val_accuracy: 0.7795
Epoch 3/10
546/546 [==============================] - 4s 7ms/step - loss: 0.4395 - accuracy: 0.7954 - val_loss: 0.4227 - val_accuracy: 0.8059
Epoch 4/10
546/546 [==============================] - 4s 7ms/step - loss: 0.3828 - accuracy: 0.8292 - val_loss: 0.3778 - val_accuracy: 0.8282
Epoch 5/10
546/546 [==============================] - 4s 7ms/step - loss: 0.3355 - accuracy: 0.8514 - val_loss: 0.3653 - val_accuracy: 0.8390
Epoch 6/10
546/546 [==============================] - 4s 7ms/step - loss: 0.2942 - accuracy: 0.8722 - val_loss: 0.3703 - val_accuracy: 0.8369
Epoch 7/10
546/546 [==============================] - 4s 7ms/step - loss: 0.2546 - accuracy: 0.8943 - val_loss: 0.3761 - val_accuracy: 0.8426
Epoch 8/10
546/54

In [30]:
#Making custom predictions
def make_prediction(test_img):
    t_img = cv2.imread(test_img)
    t_img_rz = cv2.resize(t_img, (IMG_SIZE, IMG_SIZE))
    t_img_rs = t_img_rz.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    result = model.predict(t_img_rs)
    return result